<a href="https://colab.research.google.com/github/kudothang1603/PYTHON/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import các thư viện 

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/csv /ML-EdgeIIoT-dataset.csv')
df 

đọc file và hiển thị 

In [ ]:
df.info()

*Có 62 cột , dữ liệu có 42 feature kiểu float, 1 feature int64, 20 feature kiểu object.
*Dữ liệu không bị thiếu 

In [ ]:
df.columns

In ra các cột của tệp 

In [ ]:
from sklearn.utils import shuffle
drop_columns = ["frame.time", "ip.src_host", "ip.dst_host", "arp.src.proto_ipv4","arp.dst.proto_ipv4", 

         "http.file_data","http.request.full_uri","icmp.transmit_timestamp",

         "http.request.uri.query", "tcp.options","tcp.payload","tcp.srcport",

         "tcp.dstport", "udp.port", "mqtt.msg"]

df.drop(drop_columns, axis=1, inplace=True)
df 

In [ ]:
df.describe()

in ra các cột có kiểu dữ liệu là số 

In [ ]:
df["Attack_type"].value_counts().plot(kind="bar")

In [68]:
df1 = df.copy()

In [69]:
df1.isna().sum()[df1.isna().sum() > 0]

Series([], dtype: int64)

Kiểm tra giá trị bị thiếu 

In [70]:
df2 = df1.copy()

In [ ]:
fig, axs = plt.subplots()
sns.boxplot(data=df2)
plt.show()

Kiểm tra outliers IQR 
- không có các ngoại lai 

In [ ]:
x= df.iloc[:, :-1].values
x 

In [ ]:
y= df.iloc[:, -1].values  
y 

In [75]:
object = ['http.request.method','http.referer','http.request.version','dns.qry.name.len',
          'mqtt.conack.flags','mqtt.protoname','mqtt.topic','Attack_type']
number = ['arp.opcode','arp.hw.size','icmp.checksum','icmp.seq_le', 'icmp.unused', 'http.content_length','http.response','http.tls_port',
          'tcp.ack', 'tcp.ack_raw', 'tcp.checksum', 'tcp.connection.fin','tcp.connection.rst','tcp.connection.syn','tcp.connection.synack',
          'tcp.flags','tcp.flags.ack','tcp.len', 'tcp.seq','udp.stream', 'udp.time_delta','dns.qry.name','dns.qry.qu', 'dns.qry.type','dns.retransmission',
          'dns.retransmit_request','dns.retransmit_request_in','mqtt.conflag.cleansess','mqtt.conflags','mqtt.hdrflags','mqtt.len','mqtt.msg_decoded_as',
          'mqtt.msgtype','mqtt.proto_len','mqtt.topic_len','mqtt.ver','mbtcp.len','mbtcp.trans_id','mbtcp.unit_id','Attack_label']

In [ ]:
df_number = df2.drop(object, axis = 1)
scale = MinMaxScaler()
scale.fit(df_number)
df_scale = scale.transform(df_number)
df_scale = pd.DataFrame(df_scale, columns = number)

df3 = df2.drop(number, axis=1)
df3 = pd.DataFrame(df3.values, columns = df3.columns)
df3[number] = df_scale[number]
df3

In [77]:
from sklearn.preprocessing import LabelEncoder

df_object = df3.drop(number, axis = 1)
encoder = LabelEncoder()
for i in df_object:
    df_object[i] = encoder.fit_transform(df_object[i].astype(str))


In [ ]:
df5 = df3.copy()
df5.drop(object, axis = 1)
df5[object] = df_object[object]
df5

In [79]:
df6 = df5.copy()

In [80]:
x = df6.drop(['Attack_type'], axis = 1)
y = df6.Attack_type 
x.shape, y.shape

((157800, 47), (157800,))

In [81]:
pca = PCA(n_components=8)
pca.fit(x)
x_pca = pca.transform(x)

In [82]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
smote = RandomOverSampler(random_state=0)

x_im, y_im = smote.fit_resample(x_pca, y)

In [83]:
x_train, x_test, y_train, y_test = train_test_split(x_im, y_im, test_size = 0.2, random_state = 0, shuffle=True, stratify=y_im)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((291612, 8), (291612,), (72903, 8), (72903,))

**Training Machine Learning Model** 

In [84]:
from sklearn.tree import DecisionTreeRegressor 
dt_model = DecisionTreeRegressor(random_state = 1)

In [85]:
#fit training data into model 
dt_model.fit(x_train, y_train)

DecisionTreeRegressor(random_state=1)

In [87]:
y_preds = dt_model.predict(x_test)

In [88]:
y_preds 

array([12.,  2.,  3., ...,  9.,  1., 11.])

In [89]:
pd.DataFrame({'y': y_test, 'y_preds': y_preds})

,y,y_preds
328407,12,12.0
194538,2,2.0
46131,3,3.0
189757,2,2.0
287518,9,9.0
...,...,...
155551,2,2.0
350578,14,14.0
288599,9,9.0
172050,1,1.0
